In [1]:
fname = '../data/sqlResult_1558435.csv'

In [2]:
import pandas as pd
import numpy as np
import jieba
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
content = pd.read_csv(fname, encoding='gb18030')

In [4]:
content.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [5]:
len(content)

89611

In [6]:
xinhua_news = content[content['source'] == '新华社']

In [7]:
len(xinhua_news)/len(content)

0.8778051801676133

## 数据处理

In [8]:
def token(word):
    if word['source'] == '新华社':
#         print('find')
        label = 1
    else:
#         print('Not')
        label = 0
    return pd.Series([label, word['content']], index= ['label', 'content'])

In [9]:
content_dropna = content.dropna(subset=['source', 'content'])

In [10]:
len(content_dropna)

87052

In [11]:
data = content_dropna.apply(token, axis=1)

In [12]:
data.head()

,label,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [13]:
corpus = data.content.to_list()
y = data.label.values.astype(np.int)

In [14]:
y.shape, len(corpus)

((87052,), 87052)

In [15]:
corpus[0:3]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n',
 '此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n至于电池缩水，可能与刘作虎所说，一加手机5要做市面最轻薄大屏旗舰的设定有关。\r\n按照目前掌握的资料，一加手机5拥有5.5寸1080P三星AMOLED显示屏、6G/8GB RAM，64GB/128GB ROM，双1600万摄像头，备货量“惊喜”。\r\n根据京东泄露的信息，一加5起售价是xx99元，应该是在2799/2899/2999中的某个。\r\n']

In [16]:
corpus_cut = []
mask = []
for sentence in tqdm(corpus):
    if not isinstance(sentence, str):
        mask.append(False)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'\w+', sentence))
    corpus_cut.append(' '.join(jieba.cut(sentence)))
len(corpus_cut)

  0%|                                                                                        | 0/87052 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\60230\AppData\Local\Temp\jieba.cache
Loading model cost 0.839 seconds.
Prefix dict has been built succesfully.
100%|███████████████████████████████████████████████████████████████████████████| 87052/87052 [02:34<00:00, 564.45it/s]


87052

In [17]:
corpus_cut[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [18]:
len(mask)

87052

In [19]:
vectorizer = TfidfVectorizer(max_features = 200)
X = vectorizer.fit_transform(corpus_cut)

In [20]:
X = X.toarray()
X.shape

(87052, 200)

In [21]:
y.shape

(87052,)

# KNN

In [22]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier

In [23]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.2,random_state=24)

In [24]:
Xtrain.shape, Ytrain.shape

((69641, 200), (69641,))

In [25]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=55, test_size=0.2)

In [26]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
knc = KNeighborsClassifier(n_neighbors=5, algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1) 
knc.fit(Xtrain,Ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [27]:
knc.score(xtrain,ytrain)

0.9523082998276853

In [28]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html
# precision (y_true, y_predict)
precision_score(ytrain, knc.predict(xtrain))

0.9642134350432008

In [29]:
recall_score(ytrain, knc.predict(xtrain))

0.9837592074175552

In [30]:
f1_score(ytrain, knc.predict(xtrain))

0.9738882610191146

In [31]:
# roc_auc_score(y_true, y_score,)
# For binary y_true, y_score is supposed to be the score of the class with greater label.
roc_auc_score(ytrain, knc.predict_proba(xtrain)[:,1])

0.9799199952795499

### 调整KNC参数  k=1
注意tfidf单词维度不能选太大，不然knn巨慢（本身KNN相比LR就有点慢）

In [32]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y, random_state = 0, test_size=0.2)

In [33]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=333, test_size=0.2)

In [34]:
xtrain.shape,xvalid.shape

((55712, 200), (13929, 200))

In [35]:
# k = 1 算法调整为 brute(后来证明只能用brute)
knc = KNeighborsClassifier(n_neighbors=1, algorithm='brute', leaf_size=30, p=2, metric_params=None, n_jobs=-1) 
knc.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform')

In [36]:
knc.score(xvalid, yvalid)

0.9333046162682174

In [37]:
y_predict = knc.predict(xvalid)
y_predict_proba = knc.predict_proba(xvalid)

In [38]:
precision_score(yvalid, y_predict)

0.9602084320227381

In [39]:
recall_score(yvalid, y_predict)

0.9662350043695876

In [40]:
f1_score(yvalid, y_predict)

0.9632122916089177

In [41]:
roc_auc_score(yvalid, y_predict_proba[:,1])

0.7953380685037208

**roc明显变低**

### 算法改为 brute k--->8

In [42]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=88, test_size=0.2)

In [43]:
knc = KNeighborsClassifier(n_neighbors=8, algorithm='brute', leaf_size=30, p=2, n_jobs=-1) 
knc.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=8, p=2,
                     weights='uniform')

In [44]:
knc.score(xvalid, yvalid)

0.9184435350707157

In [45]:
y_predict = knc.predict(xvalid)
y_predict_proba = knc.predict_proba(xvalid)

In [46]:
precision_score(yvalid, y_predict)

0.9524940617577197

In [47]:
recall_score(yvalid, y_predict)

0.9573452172529047

In [48]:
f1_score(yvalid, y_predict)

0.9549134783298936

In [49]:
roc_auc_score(yvalid, y_predict_proba[:,1])

0.9096634480142938

**效果也没有第一次好**

### 这次改为计算: 曼哈顿距离 且 权重weight由uniform改为distance
* When p = 1, this is equivalent to using manhattan_distance (l1)
* ‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.
* distance : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.


In [50]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=78, test_size=0.2)

In [51]:
knc = KNeighborsClassifier(n_neighbors=8, weights='distance',algorithm='brute', leaf_size=30, p=1,n_jobs=-1) 
knc.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=8, p=1,
                     weights='distance')

In [52]:
y_predict = knc.predict(xvalid)
y_predict_proba = knc.predict_proba(xvalid)

In [53]:
precision_score(yvalid, y_predict)

0.9821599512662084

In [54]:
recall_score(yvalid, y_predict)

0.8981378322457425

In [55]:
f1_score(yvalid, y_predict)

0.9382716049382716

In [56]:
roc_auc_score(yvalid, y_predict_proba[:,1])

0.938943537330525

**precision提高，recall降低，f1略降低，roc提高**

### 提高叶子节点数目 
本来想把算法改为ball tree,kd tree（发现都用不了，因为x是稀疏矩阵），默认就是 brute算法咯

In [28]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=23, test_size=0.2)

In [29]:
knc = KNeighborsClassifier(n_neighbors=8, weights='distance',algorithm='auto', leaf_size=80, p=1,n_jobs=-1) 
knc.fit(xtrain,ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=80, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=8, p=1,
                     weights='distance')

In [30]:
y_predict = knc.predict(xvalid)
y_predict_proba = knc.predict_proba(xvalid)

In [31]:
precision_score(yvalid, y_predict)

0.9816878293167487

In [32]:
recall_score(yvalid, y_predict)

0.9041368337311058

In [33]:
f1_score(yvalid, y_predict)

0.9413177620408333

In [34]:
roc_auc_score(yvalid, y_predict_proba[:,1])

0.9434779656294142

**roc提高了，增加叶子数量有效果，但不是很明显**  
**采取第一种方案。KNN on 测试集**

In [83]:
knc = KNeighborsClassifier(n_neighbors=5, algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1) 
knc.fit(Xtrain,Ytrain)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [84]:
knc_score = knc.score(Xtest, Ytest)

In [85]:
knc_f1 =  f1_score(Ytest, knc.predict(Xtest))
knc_roc = roc_auc_score(Ytest, knc.predict(Xtest))

In [86]:
print('score: ', knc_score)
print('f1_score: ', knc_f1)
print('roc_auc_score: ', knc_roc)

score:  0.9286657859973579
f1_score:  0.960908976457258
roc_auc_score:  0.7547548788746792


# Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=888, test_size=0.2)

In [28]:
# solver : str, {‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, optional (default=’liblinear’)
clf = LogisticRegression(solver='saga').fit(xtrain, ytrain)
# 改上面几个solver几乎没变化
# 另外LR真是训练超快，好顶赞

In [29]:
clf.score(xvalid, yvalid)

0.9768827625816642

In [30]:
y_pred = clf.predict(xvalid)
y_pred_prob = clf.predict_proba(xvalid)

In [33]:
precision_score(yvalid, y_pred)

0.9824575204531152

In [35]:
recall_score(yvalid, y_pred)

0.9921353670162059

In [36]:
f1_score(yvalid, y_pred)

0.9872727272727272

In [37]:
roc_auc_score(yvalid, y_pred_prob[:,1])

0.9910985498777642

In [38]:
LR_score = clf.score(Xtest,Ytest)
LR_f1 =  f1_score(Ytest, clf.predict(Xtest))
LR_roc = roc_auc_score(Ytest, clf.predict(Xtest))

In [39]:
print('score: ', LR_score)
print('f1_score: ', LR_f1)
print('roc_auc_score: ', LR_roc)

score:  0.9754752742519097
f1_score:  0.9864946073314989
roc_auc_score:  0.9099307936565718


# Naive Bayes

In [40]:
from sklearn.naive_bayes import GaussianNB

In [41]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=233, test_size=0.2)

In [42]:
nb = GaussianNB()
nb.fit(xtrain, ytrain)

GaussianNB(priors=None, var_smoothing=1e-09)

In [43]:
y_pred = nb.predict(xvalid)
y_pred_prob = nb.predict_proba(xvalid)

In [44]:
nb.score(xvalid, yvalid)

0.795247325723311

In [45]:
precision_score(yvalid, y_pred)

0.9948291594849438

In [47]:
recall_score(yvalid, y_pred)

0.7779275350828511

In [48]:
f1_score(yvalid, y_pred)

0.8731090941448657

In [49]:
roc_auc_score(yvalid, y_pred_prob[:,1])

0.9141807000882237

In [50]:
NB_score = nb.score(Xtest,Ytest)
NB_f1 =  f1_score(Ytest, nb.predict(Xtest))
NB_roc = roc_auc_score(Ytest, nb.predict(Xtest))

In [51]:
print('score: ', NB_score)
print('f1_score: ', NB_f1)
print('roc_auc_score: ', NB_roc)

score:  0.79593360519212
f1_score:  0.8732746014195528
roc_auc_score:  0.8711056406336148


# SVM

In [54]:
from sklearn.svm import SVC, LinearSVC

In [55]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=333, test_size=0.2)

In [62]:
svc=SVC(probability=True,gamma='scale')
svc.fit(xtrain, ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [63]:
svc.score(xvalid, yvalid)

0.9843491995118099

In [64]:
y_pred = svc.predict(xvalid)
y_pred_prob = svc.predict_proba(xvalid)

In [65]:
precision_score(yvalid, y_pred)

0.9893810920041207

In [66]:
recall_score(yvalid, y_pred)

0.9933168907629883

In [67]:
f1_score(yvalid, y_pred)

0.9913450849610926

In [68]:
roc_auc_score(yvalid, y_pred_prob[:,1])

0.994458559340675

In [70]:
svc_score = svc.score(Xtest, Ytest)
svc_f1 = f1_score(Ytest, svc.predict(Xtest))
svc_roc = roc_auc_score(Ytest, svc.predict(Xtest))

In [71]:
print('score: ', svc_score)
print('f1_score: ', svc_f1)
print('roc_auc_score: ', svc_roc)

score:  0.9820228591120556
f1_score:  0.9900732612349752
roc_auc_score:  0.9402085973458072


# Decision Tree

In [72]:
from sklearn.tree import DecisionTreeClassifier

In [73]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(xtrain, ytrain)
y_pred = decision_tree.predict(xvalid)
y_pred_prob = decision_tree.predict_proba(xvalid)
print('score: ', decision_tree.score(xvalid, yvalid))
print('recall score: ', recall_score(yvalid, y_pred))
print('f1_score: ', f1_score(yvalid, y_pred))
print('roc_auc_score: ', roc_auc_score(yvalid, y_pred))

score:  0.9852825041280782
recall score:  0.9922030392234864
f1_score:  0.9918479341472144
roc_auc_score:  0.9567632843176256


### 测试集数据

In [74]:
# test score
decision_tree.score(Xtest, Ytest)

0.9861007409109184

In [75]:
decision_tree_score = decision_tree.score(Xtest, Ytest)
decision_tree_f1 = f1_score(Ytest, decision_tree.predict(Xtest))
decision_tree_roc = roc_auc_score(Ytest, decision_tree.predict(Xtest))

In [76]:
print('score: ', decision_tree_score)
print('f1_score: ', decision_tree_f1)
print('roc_auc_score: ', decision_tree_roc)

score:  0.9861007409109184
f1_score:  0.9923081812980739
roc_auc_score:  0.9611234285332927


# Random Forest

In [77]:
from sklearn.ensemble import RandomForestClassifier

In [78]:
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, random_state=53, test_size=0.2)

In [79]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(xtrain, ytrain)
y_pred = random_forest.predict(xvalid)
y_pred_prob = random_forest.predict_proba(xvalid)
print('score: ', random_forest.score(xvalid, yvalid))
print('recall score: ', recall_score(yvalid, y_pred))
print('f1_score: ', f1_score(yvalid, y_pred))
print('roc_auc_score: ', roc_auc_score(yvalid, y_pred))

score:  0.992102807093115
recall score:  0.9969896221183554
f1_score:  0.9956487341772152
roc_auc_score:  0.9709297268325314


### 使用测试集

In [80]:
# test score
random_forest_score = random_forest.score(Xtest, Ytest)
random_forest_f1 = f1_score(Ytest, random_forest.predict(Xtest))
random_forest_roc = roc_auc_score(Ytest, random_forest.predict(Xtest))

In [81]:
print('score: ', random_forest_score)
print('f1_score: ', random_forest_f1)
print('roc_auc_score: ', random_forest_roc)

score:  0.9905232324392625
f1_score:  0.9947637332994828
roc_auc_score:  0.967301946089506


In [88]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes',  
              'Decision Tree'],
    'Score': [svc_score, knc_score, LR_score, 
              random_forest_score, NB_score, 
             decision_tree_score],
    'F1_score':[ svc_f1, knc_f1, LR_f1,
                random_forest_f1,NB_f1,
                decision_tree_f1
    ],
    'ROC_score':[ svc_roc, knc_roc, LR_roc, 
                 random_forest_roc, NB_roc,
                 decision_tree_roc
    ]

})
models.sort_values(by='Score', ascending=False)

,Model,Score,F1_score,ROC_score
3,Random Forest,0.990523,0.994764,0.967302
5,Decision Tree,0.986101,0.992308,0.961123
0,Support Vector Machines,0.982023,0.990073,0.940209
2,Logistic Regression,0.975475,0.986495,0.909931
1,KNN,0.928666,0.960909,0.754755
4,Naive Bayes,0.795934,0.873275,0.871106


**因此选择random forest**

## 预测为1，但实际为0，可能为抄袭者（也可能预测错了）

In [89]:
y_pred = random_forest.predict(X)
y_pred.shape,y.shape

((87052,), (87052,))

In [91]:
len(content_dropna)

87052

In [92]:
content_dropna['y'] = y
content_dropna['y_pred'] = y_pred
content_dropna.head(2)

H:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
H:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,author,source,content,feature,title,url,y,y_pred
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0,0
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0,0


In [97]:
fake_news = content_dropna[(content_dropna.y == 0) & (content_dropna.y_pred == 1)]
len(fake_news)
# 算法认为抄袭的有180篇新闻

180

In [98]:
copy_sources = set(fake_news.source.to_list())
# 抄袭来源有多少种

In [99]:
len(copy_sources)

77